In [1]:
import os
base_directory = '/workspaces/streaming-prj'
os.chdir(base_directory)

make a connection

In [3]:
import socket

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5005  # Port to use for communication

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket

# Message to send
message = "Hello from Notebook 1"

# Send the message
sock.sendto(message.encode(), (UDP_IP, UDP_PORT))
print(f"Sent message: {message}")
sock.close()


Sent message: Hello from Notebook 1


send larger content

In [4]:
import socket
import json
import math

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5005  # Port to use for communication

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket

# Original message
message = "This is a much larger message. " * 50  # Simulate a larger message

# Convert the message to a JSON format
data = {"message": message}
json_data = json.dumps(data)

# Split the JSON data into smaller chunks
chunk_size = 512  # Define chunk size for transmission
chunks = [json_data[i:i + chunk_size] for i in range(0, len(json_data), chunk_size)]

# Send each chunk
for idx, chunk in enumerate(chunks):
    sock.sendto(chunk.encode(), (UDP_IP, UDP_PORT))
    print(f"Sent chunk {idx+1}/{len(chunks)}: {chunk[:50]}...")  # Show first 50 characters
sock.close()


Sent chunk 1/4: {"message": "This is a much larger message. This i...
Sent chunk 2/4: s is a much larger message. This is a much larger ...
Sent chunk 3/4: er message. This is a much larger message. This is...
Sent chunk 4/4:  is a much larger message. "}...


send an image

In [6]:
import socket
import os

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5005  # Port to use for communication

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket

# Path to the binary file (e.g., an image or audio file)
file_path = "image/example_image.jpg"  # Change this to any binary file

# Open the file in binary mode
with open(file_path, "rb") as file:
    # Read the file in chunks
    chunk_size = 1024  # 1 KB chunks
    chunk_number = 0
    while chunk := file.read(chunk_size):
        sock.sendto(chunk, (UDP_IP, UDP_PORT))
        chunk_number += 1
        print(f"Sent chunk {chunk_number} of size {len(chunk)} bytes")

# Send an 'END' signal to indicate the end of the transmission
sock.sendto(b'END', (UDP_IP, UDP_PORT))
print("Sent termination signal: 'END'")

sock.close()


Sent chunk 1 of size 1024 bytes
Sent chunk 2 of size 1024 bytes
Sent chunk 3 of size 1024 bytes
Sent chunk 4 of size 1024 bytes
Sent chunk 5 of size 1024 bytes
Sent chunk 6 of size 1024 bytes
Sent chunk 7 of size 1024 bytes
Sent chunk 8 of size 1024 bytes
Sent chunk 9 of size 1024 bytes
Sent chunk 10 of size 1024 bytes
Sent chunk 11 of size 1024 bytes
Sent chunk 12 of size 1024 bytes
Sent chunk 13 of size 1024 bytes
Sent chunk 14 of size 1024 bytes
Sent chunk 15 of size 1024 bytes
Sent chunk 16 of size 1024 bytes
Sent chunk 17 of size 1024 bytes
Sent chunk 18 of size 1024 bytes
Sent chunk 19 of size 1024 bytes
Sent chunk 20 of size 1024 bytes
Sent chunk 21 of size 1024 bytes
Sent chunk 22 of size 1024 bytes
Sent chunk 23 of size 1024 bytes
Sent chunk 24 of size 1024 bytes
Sent chunk 25 of size 1024 bytes
Sent chunk 26 of size 1024 bytes
Sent chunk 27 of size 1024 bytes
Sent chunk 28 of size 1024 bytes
Sent chunk 29 of size 1024 bytes
Sent chunk 30 of size 1024 bytes
Sent chunk 31 of si

test audio

In [9]:
import numpy as np
import wave
import pyaudio

# Parameters for sine wave
sample_rate = 44100  # Samples per second
duration = 2  # Duration of the tone in seconds
frequency = 440  # Frequency of the sine wave (Hz, A4 pitch)

# Generate the sine wave
t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)  # Time values
audio_data = np.sin(2 * np.pi * frequency * t)  # Sine wave formula

# Convert to 16-bit PCM format
audio_data = np.int16(audio_data * 32767)  # Scale to 16-bit PCM range

# Save as .wav file (optional)
with wave.open("sine_wave.wav", 'wb') as wf:
    wf.setnchannels(1)  # Mono audio
    wf.setsampwidth(2)  # 16-bit audio
    wf.setframerate(sample_rate)
    wf.writeframes(audio_data.tobytes())

# Play the generated sine wave using PyAudio
p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paInt16,
                channels=1,
                rate=sample_rate,
                output=True)

# Play audio in chunks
chunk_size = 1024
for i in range(0, len(audio_data), chunk_size):
    stream.write(audio_data[i:i + chunk_size].tobytes())

# Close stream
stream.stop_stream()
stream.close()
p.terminate()


ModuleNotFoundError: No module named 'pyaudio'

In [10]:
import socket
import pyaudio

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5005  # Port to use for communication

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket

# Setup PyAudio for real-time audio capture
p = pyaudio.PyAudio()

# Audio settings
FORMAT = pyaudio.paInt16  # Audio format (16-bit PCM)
CHANNELS = 1  # Mono
RATE = 44100  # Sample rate (Hz)
CHUNK = 1024  # Number of frames per buffer

stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)

print("Streaming audio...")

# Capture and send audio in real-time
while True:
    audio_data = stream.read(CHUNK)  # Capture a chunk of audio

    # Send the audio chunk over UDP
    sock.sendto(audio_data, (UDP_IP, UDP_PORT))

    # To stop, we could add a condition (e.g., key press or timer), for now, we let it run indefinitely
    # For example: if condition_to_stop: break
    
# Send the "END" signal when finished
sock.sendto(b'END', (UDP_IP, UDP_PORT))
print("Sent termination signal: 'END'")

sock.close()
stream.stop_stream()
stream.close()
p.terminate()


ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM sysdefault
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_concat returned error

OSError: [Errno -9996] Invalid input device (no default output device)

In [ ]:
m,,,,,asdfasdf